In [5]:
pip install selenium


   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.4 MB 660.6 kB/s eta 0:00:15
   ---------------------------------------- 0.1/9.4 MB 465.5 kB/s eta 0:00:21
    --------------------------------------- 0.1/9.4 MB 722.1 kB/s eta 0:00:13
    --------------------------------------- 0.2/9.4 MB 706.2 kB/s eta 0:00:14
   - -------------------------------------- 0.3/9.4 MB 983.0 kB/s eta 0:00:10
   - -------------------------------------- 0.4/9.4 MB 1.3 MB/s eta 0:00:08
   -- ------------------------------------- 0.5/9.4 MB 1.5 MB/s eta 0:00:06
   -- ------------------------------------- 0.6/9.4 MB 1.7 MB/s eta 0:00:06
   --- ------------------------------------ 0.7/9.4 MB 1.6 MB/s eta 0:00:06
   --- ------------------------------------ 0.9/9.4 MB 1.8 MB/s eta 0:00:05
   ---- ----------------------------------- 1.0/9.4 MB 1.8 MB/s eta 0:00:05
   ---- -------

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
conda-repo-cli 1.0.75 requires requests_mock, which is not installed.
conda-repo-cli 1.0.75 requires clyent==1.2.1, but you have clyent 1.2.2 which is incompatible.
conda-repo-cli 1.0.75 requires python-dateutil==2.8.2, but you have python-dateutil 2.9.0.post0 which is incompatible.
s3fs 2023.10.0 requires fsspec==2023.10.0, but you have fsspec 2024.3.1 which is incompatible.
spyder 5.4.3 requires jedi<0.19.0,>=0.17.2, but you have jedi 0.19.0 which is incompatible.
streamlit 1.31.1 requires packaging<24,>=16.8, but you have packaging 24.0 which is incompatible.

[notice] A new release of pip is available: 24.0 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import pandas as pd
import time


# Set up Chrome options
chrome_options = Options()
chrome_options.add_argument("--headless")  # Run in headless mode
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# Specify the path to the ChromeDriver
chrome_service = Service(r'C:\Users\Pranav\Downloads\chromedriver-win64\chromedriver-win64')  # Update this path for Windows
# For macOS/Linux, it might be: '/path/to/chromedriver'

# Initialize the WebDriver
driver = webdriver.Chrome(service=chrome_service, options=chrome_options)

# Open the Amazon search page for Intel products
url = 'https://www.amazon.in/s?k=intel&ref=nb_sb_noss'
driver.get(url)

# Allow time for the page to load
time.sleep(3)

# Collect product links
product_links = []
products = driver.find_elements(By.XPATH, '//a[@class="a-link-normal a-text-normal"]')
for product in products:
    product_links.append(product.get_attribute('href'))

reviews = []

# Visit each product page and scrape reviews
for link in product_links[:5]:  # Limiting to first 5 products for demonstration
    driver.get(link)
    time.sleep(3)
    
    # Click on the reviews section (if present)
    try:
        reviews_button = driver.find_element(By.XPATH, '//a[@data-hook="see-all-reviews-link-foot"]')
        reviews_button.click()
        time.sleep(3)
    except:
        continue
    
    # Scrape reviews from the reviews page
    while True:
        review_elements = driver.find_elements(By.XPATH, '//div[@data-hook="review"]')
        for review in review_elements:
            try:
                product_name = driver.find_element(By.ID, 'productTitle').text
                review_title = review.find_element(By.XPATH, './/a[@data-hook="review-title"]').text
                review_text = review.find_element(By.XPATH, './/span[@data-hook="review-body"]').text
                rating = review.find_element(By.XPATH, './/i[@data-hook="review-star-rating"]').text.split()[0]
                date = review.find_element(By.XPATH, './/span[@data-hook="review-date"]').text
                reviewer = review.find_element(By.XPATH, './/span[@class="a-profile-name"]').text
                reviews.append({
                    'product_name': product_name,
                    'review_title': review_title,
                    'review_text': review_text,
                    'rating': rating,
                    'date': date,
                    'reviewer': reviewer
                })
            except Exception as e:
                print(f"Error: {e}")
                continue
        
        # Move to the next page of reviews
        try:
            next_button = driver.find_element(By.XPATH, '//li[@class="a-last"]/a')
            next_button.click()
            time.sleep(3)
        except:
            break

# Convert to DataFrame and save to CSV
df = pd.DataFrame(reviews)
df.to_csv('intel_reviews.csv', index=False)

# Close the WebDriver
driver.quit()


In [6]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = 'https://www.amazon.in/s?k=intel&ref=nb_sb_noss'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

reviews = []
for review in soup.find_all('div', class_='review'):
    product_name = review.find('h2', class_='product-name').text
    review_title = review.find('h3', class_='review-title').text
    review_text = review.find('p', class_='review-text').text
    rating = int(review.find('span', class_='rating').text)
    date = review.find('span', class_='date').text
    reviewer = review.find('span', class_='reviewer').text
    
    reviews.append({
        'product_name': product_name,
        'review_title': review_title,
        'review_text': review_text,
        'rating': rating,
        'date': date,
        'reviewer': reviewer
    })

# Convert to DataFrame
df = pd.DataFrame(reviews)
df.to_csv('intell.csv', index=False)
print("intel file successfully scrapped")


intel file successfully scrapped
